In [96]:
import pyodbc
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import precision_score
import pickle
from datetime import datetime
import warnings

def getData(symbol):
    conn = pyodbc.connect("DRIVER={SQL Server};SERVER=lenovo-desktop;DATABASE=Qihuo;UID=samtsql;PWD=F(W}q:TsyK,7^+>`P28e79s#Uc5n")
    # conn = pyodbc.connect("DRIVER={SQL Server};SERVER=.;Database=Qihuo;Trusted_Connection=True")    
    query = f"""select m1.date as 'Date',DATEPART(hour, m1.Date) AS 'Hour', m1.[Close] as 'Market', m2.[close] as 'Stock'  from MinuteQuote m1
    inner join MinuteQuote m2 on m1.Date = m2.date
    where m1.Date > '2002-04-08 01:20:00' and m1.Contract = 'ym' and m2.Contract = '{symbol}'

    order by m1.date"""

    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)
        df = pd.read_sql(query, conn)
        df.set_index('Date', inplace=True)
        return df


In [97]:
def transformData(df):
    df[f'StockReturn'] = df["Stock"] / df["Stock"].shift(1) - 1
    df[f'MarketReturn'] = df["Market"] / df["Market"].shift(1) - 1
    df[f"OutPerform"] = df['StockReturn'] - df['MarketReturn']
    df["Target"] = (df.apply(lambda x: x > 0)["OutPerform"]).astype(int)


In [98]:
def getPredictors(df):
    predictors = ['Hour']
    for i in range(12):
        df[f'OutPerform{pow(2,i)*5}'] = df['OutPerform'].rolling(pow(2,i)).sum()
        predictors.append(f'OutPerform{pow(2,i)*5}')
        df[f'StockReturn{pow(2,i)*5}'] = df['StockReturn'].rolling(pow(2,i)).sum()
        predictors.append(f'StockReturn{pow(2,i)*5}')
        df[f'MarketReturn{pow(2,i)*5}'] = df['MarketReturn'].rolling(pow(2,i)).sum()
        predictors.append(f'MarketReturn{pow(2,i)*5}')
        # df[f'MarketReturn{i}'] = df["Market"] / df["Market"].shift(pow(2,i)) - 1
        # predictors.append(f'OutPerform{i}')
        # df[f'StockReturn{i}'] = df["Stock"] / df["Stock"].shift(pow(2,i)) - 1
        # predictors.append(f'StockReturn{i}')
        # df[f"OutPerform{i}"] = df[f'StockReturn{i}'] - df[f'MarketReturn{i}']
        # df.drop([f'MarketReturn{i}'], axis = 1)
    return predictors


In [99]:
def getMLdata(df, predictors):
    df = df[(df.index.minute == 25) | (df.index.minute == 55) ]
    prev = df.copy()
    prev = prev.shift(1)
    data = df[["OutPerform","Target","Stock","Market"]]
    data = data.join(prev[predictors])
    transformData(data)
    data = data.copy().dropna()
    # print(data)
    return data


In [100]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [101]:
def backtest(data, model, predictors, start=20000, step=5000):
    all_predictions = []
    loop = 0
    for i in range(start, data.shape[0], step):
        train = data.iloc[loop * step:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
        loop = loop + 1

    # print(all_predictions)    
    return pd.concat(all_predictions)


In [102]:
model = RandomForestClassifier(n_estimators=100, min_samples_split=1200, random_state=1)
symbols = [
'CVX'
,'HON',
'CRM',
'UNH',
'CSCO',
'WMT',
'AXP',
'JPM',
'MCD',
'HD',
'AMGN',
'V',
'INTC',
'WBA',
'GS',
'JNJ',
'PG',
'AAPL',
'DIS',
'MMM',
'MRK',
'MSFT',
'TRV',
'VZ',
'IBM',
'CAT',
'NKE'
]
for s in symbols:
    df = getData(s)
    transformData(df)
    predictors = getPredictors(df)
    data = getMLdata(df, predictors)
    predictions = backtest(data, model, predictors)
    pickle.dump(model, open(f"Model/{s}", 'wb'))
    tempData = data[['OutPerform']].copy()
    result = pd.merge(tempData, predictions, left_index=True, right_index=True)
    result.to_csv(f'./Result/{s}.csv')

    # precision_score(predictions["Target"], predictions["Predictions"])
    

In [106]:
import glob
files = [f for f in glob.glob("Result/*.csv")]
result = pd.DataFrame(columns = ['symbol', 'year', 'gain','count'])
for file in files:

    s = file.split('\\')[1].replace('.csv','')
    df = pd.read_csv(f'Result/{s}.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df['year'] = df.index.strftime('%Y')
    groupbyyear = df.groupby('year')

    for group_name, df_group in groupbyyear:
        count = 0
        newtrade = True
        gsum = 0
        for ind in df_group.index:
            if df_group['Predictions'][ind] >.57:
                gsum = gsum + df_group['OutPerform'][ind]
                if newtrade == True:
                    count = count + 1
                    newtrade = False
            elif df_group['Predictions'][ind] >= .50 and newtrade == False:
                gsum = gsum + df_group['OutPerform'][ind]
            else:
                newtrade = True 

        if count > 0:       
            result.loc[len(result.index)] = [s, group_name, gsum, count]

dfgain = result.groupby('year')['gain'].sum()
dfcount = result.groupby('year')['count'].sum()
dfresult = dfgain.to_frame().merge(dfcount.to_frame(), left_index=True, right_index=True)
dfresult['avg'] = dfresult['gain']/dfresult['count']
print(dfresult)
print(dfresult['count'].sum())
print(dfresult['gain'].sum()/dfresult['count'].sum())            

          gain  count       avg
year                           
2008  2.857302   1195  0.002391
2009  2.400535   1491  0.001610
2010  0.721387    460  0.001568
2011  0.755342    539  0.001401
2012  0.175746    517  0.000340
2013  0.396035    591  0.000670
2014  0.242678    553  0.000439
2015  0.827526    754  0.001098
2016  0.191379    670  0.000286
2017  0.018756    521  0.000036
2018  0.846573    682  0.001241
2019  0.226003    475  0.000476
2020  1.208406    772  0.001565
2021  0.342746    254  0.001349
2022  0.126617    221  0.000573
9695
0.0011693688143741773


In [104]:
dfgain = result.groupby('symbol')['gain'].sum()
dfcount = result.groupby('symbol')['count'].sum()
dfresult = dfgain.to_frame().merge(dfcount.to_frame(), left_index=True, right_index=True)
dfresult['avg'] = dfresult['gain']/dfresult['count']
print(dfresult)
print(dfresult['count'].sum())
print(dfresult['gain'].sum()/dfresult['count'].sum())  

            gain  count       avg
symbol                           
AAPL   -1.086187   1535 -0.000708
AMGN    0.026724    133  0.000201
AXP     0.072559    371  0.000196
CAT    -0.168051    294 -0.000572
CRM    -0.053530     65 -0.000824
CSCO   -2.048686   1876 -0.001092
CVX    -0.303305    377 -0.000805
DIS    -0.320291    213 -0.001504
GS     -0.266318    377 -0.000706
HD     -0.262852    299 -0.000879
HON    -0.390189    395 -0.000988
IBM    -0.231808    474 -0.000489
INTC   -1.570902   1989 -0.000790
JNJ    -0.250947    156 -0.001609
JPM    -0.301746    428 -0.000705
MCD    -0.301896    298 -0.001013
MMM    -0.601496    467 -0.001288
MRK    -0.284364    156 -0.001823
MSFT   -0.277094    502 -0.000552
NKE    -0.488564    304 -0.001607
PG     -0.133337    301 -0.000443
TRV    -1.023249    608 -0.001683
UNH     0.200078    188  0.001064
V      -0.298829    168 -0.001779
VZ     -0.084304    194 -0.000435
WBA     0.157509    343  0.000459
WMT    -0.019367    353 -0.000055
12864
-0.00080

In [105]:

result

,symbol,year,gain,count
0,AAPL,2008,-0.278709,131
1,AAPL,2009,-0.025825,165
2,AAPL,2010,-0.151806,220
3,AAPL,2011,-0.007332,183
4,AAPL,2012,-0.176329,230
...,...,...,...,...
372,WMT,2018,-0.005161,19
373,WMT,2019,-0.016466,25
374,WMT,2020,-0.070269,30
375,WMT,2021,0.042304,24
